## ***Add this to yolov7 folder to work !!***

# Data preprocessing

In [ ]:
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
import pandas as pd
import torch
from models.experimental import attempt_load
from utils.general import non_max_suppression, xyxy2xywh
from utils.torch_utils import time_synchronized

In [ ]:
df = pd.read_csv('../../frameRecorder/data.csv')

In [ ]:
df.head()

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if(torch.cuda.is_available()):
    torch.cuda.empty_cache()

print(torch.cuda.get_device_name(0))

In [ ]:
# Load model
model = attempt_load('yolov7.pt', map_location=device)

In [ ]:
# Set parameters
if(torch.cuda.is_available()):
    model.half()
img_size = (128, 128)
conf_thres = 0.3

if(device == 'cuda'):
    model(torch.zeros(1, 3, img_size).to(device).type_as(next(model.parameters())))

In [ ]:
def detect_objects_per_frame(frame):
    torch.cuda.empty_cache()

    # Resize image
    img = cv.resize(frame, img_size)
    tempimg = img
    img = img.transpose(2, 0, 1)  # HWC to CHW
    img = torch.from_numpy(img).to(device).float()
    img = img.half()
    img /= 255.0

    # Detect objects
    t1 = time_synchronized()
    # to prevent memory leaks !!!
    with torch.no_grad():
        pred = model(img.unsqueeze(0), augment=False)[0]
    pred = non_max_suppression(pred, conf_thres, agnostic=False)
    t2 = time_synchronized()

    img = img.detach()
    del img

    # dictionary to story informations about objects : x, y, width and height. The key is the name of the object.
    objs = {}

    # Process detections    
    for i, det in enumerate(pred):
        # Rescale detection coordinates
        if det is not None and len(det):

            # getting the x, y coordinates with the width and height and drawing the boxes
            for *xyxy, conf, cls in reversed(det):
                temp_tensor = torch.tensor(xyxy).view(1, 4)
                x, y, w, h = xyxy2xywh(temp_tensor).squeeze(0)

                # freeing memory occupied by the tensor
                temp_tensor = temp_tensor.detach()
                del temp_tensor
                torch.cuda.empty_cache()
                
                xx, yy, ww, hh = (x / img_size[0]) * frame.shape[1], (y / img_size[1]) * frame.shape[0], (w / img_size[0]) * frame.shape[1], (h / img_size[1]) * frame.shape[0]
                label = f'{model.names[int(cls)]} {conf:.2f}'
                objs[label] = [xx, yy, ww, hh]
                cv.rectangle(frame, (int(xx - ww / 2), int(yy - hh / 2)), (int(xx - ww / 2 + ww), int(yy - hh / 2 + hh)), (255, 255, 0), 2)
                cv.putText(frame, label, (int(xx - ww / 2), int(yy - h / 2) - 10), cv.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 1)


    return (objs, frame, t1, t2, tempimg)

In [ ]:
# Load image
temp = df.iloc[8000]['image_id']
img0 = cv.imread(f'../../frameRecorder/screenShots/{temp}.png')  # BGR
img0 = cv.cvtColor(img0, cv.COLOR_BGR2RGB)

# crop unnecessary data (this will change for other tasks, currently we only need to detect objects on the road) 
img0 = img0[50:490, :]


%matplotlib inline
plt.imshow(img0)

In [ ]:
# detect objects using the funtction and get the new frame, the coordinates of the objects and the time needed for the prediction only not the whole function
objectss, nframe, t1, t2, tp = detect_objects_per_frame(img0)

In [ ]:
# print the dictionary of objects and informations about their coordinates
for k, v in objectss.items():
    print(f'{k}: ', end="[")
    for i in v:
        print(f'{i}', end=", ")
    print("]")

# print time needed only for detection (this might change a lot if we use cuda instead of the CPU)
print(t2 - t1)

%matplotlib inline
plt.imshow(tp)